In [1]:
# | default_exp classes.DomoApplication_Job

A job refers to a scheduled task in an application.

For the purposes of this beta class, the only Job type implemented has been Watchdog.  This class will be heavily revised in a future development sprint.

An additional Job type might be PDP Automation Toolkit.

In [2]:
# | exporti
from dataclasses import dataclass, field
from typing import List, Optional
from enum import Enum

import httpx

from nbdev.showdoc import patch_to
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import datetime  as dt

from pprint import pprint

import domolibrary.routes.application as application_routes

import domolibrary.utils.convert as cc

In [3]:
# |hide
import os

In [4]:
# | export


@dataclass
class DomoTrigger_Schedule:
    schedule_text: str = None
    schedule_type: str = "scheduleTriggered"

    minute: int = None
    hour: int = None
    minute_str: str = None
    hour_str: str = None

    @classmethod
    def _from_str(cls, s_text, s_type):
        sched = cls(schedule_type=s_type, schedule_text=s_text)

        try:
            parsed_hour = s_text.split(" ")[2]
            parsed_minute = s_text.split(" ")[1]

            if "*" in parsed_hour or "/" in parsed_hour:
                sched.hour_str = parsed_hour
            else:
                sched.hour = int(float(parsed_hour))
            if "*" in parsed_minute:
                sched.minute_str = parsed_minute
            else:
                sched.minute = int(float(parsed_minute))

            return sched

        except Exception as e:
            print(f"unable to parse schedule {s_text}")
            print(e)

    def to_obj(self):
        return {"hour": int(self.hour), "minute": int(self.minute)}

    def to_json(self):
        minute = self.minute_str if self.minute_str is not None else str(self.minute)
        hour = self.hour_str if self.hour_str is not None else str(self.hour)
        return {
            "eventEntity": f"0 {minute} {hour} ? * *",
            # old value on Jan 13
            # "eventEntity": f'0 {minute} {hour} 1/1 * ? *',
            "eventType": self.schedule_type,
        }


@dataclass
class DomoTrigger:
    id: str
    job_id: str
    schedule: List[DomoTrigger_Schedule] = None

    @classmethod
    def _from_json(cls, obj):
        return cls(
            id=obj["triggerId"],
            job_id=obj["jobId"],
            schedule=DomoTrigger_Schedule._from_str(
                s_text=obj.get("eventEntity"), 
                s_type=obj.get("eventType")
            ),
        )


In [5]:

class DomoJob_TypeEnum(Enum):
    PDP_AUTOMATION = "Toolkit: PDP Automation"
    SCHEMA_MANAGEMENT = "Toolkit: Schema Management"
    DATASET_S3_BACKUP = "Toolkit: DataSet S3 Backup"
    USER_AUTOMATION = "Toolkit: User Automation"
    GROUP_MANAGEMENT = "Toolkit: Group Management"
    TRIGGERED_REPORTS = "Toolkit: Triggered Reports"
    REMOTE_DOMO_STATS = "Toolkit: Remote Domo Stats"
    DATASET_TAG_AUTOMATION = "Toolkit: DataSet Tag Automation"
    GEOCODER_EXECUTOR = "Geocoder Executor"
    VIRTUAL_DATASET_SHARING = "Toolkit: Virtual Dataset Sharing"
    DATA_COPY = "Toolkit: Data Copy"
    DATA_WATCHDOG = "Toolkit: Data Watchdog"


In [6]:
#| hide
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-alpha",
    domo_access_token=os.environ["ALPHA_ACCESS_TOKEN"],
)

res = await application_routes.get_applications(auth=token_auth)

txt = [f"{cc.convert_str_to_snake_case(name, is_only_alphanumeric=True).upper().replace('TOOLKIT_', '')} = \"{name}\"" 
       for name in [response['name'] for response in res.response]]

# '/n'.join(txt)

In [7]:
# | export


@dataclass
class DomoJob:
    """
    the base class only captures attributes applicable to all jobs (i.e. does not destructure execution_payload onto the class)
    build Application / Job extensions by creating children of the DomoJob class
    """
    auth: dmda.DomoAuth = field(repr=False)

    id: str
    name: str
    user_id: str
    application_id: str
    customer_id: str
    execution_timeout: int

    execution_payload: dict
    share_state: dict
    created_dt: dt.datetime
    updated_dt: dt.datetime

    is_enabled: bool  # based on triggers

    description: str = None
    accounts: List[str] = None
    triggers: List[DomoTrigger] = field(default_factory=[])

    def _convert_API_res_to_DomoJob_base_obj(obj) -> dict:
        triggers_ls = obj.get("triggers")

        return {
            "id": obj["jobId"],
            "name": obj["jobName"],
            "user_id": obj["userId"],
            "application_id": obj["applicationId"],
            "customer_id": obj["customerId"],
            "execution_timeout": obj["executionTimeout"],
            "execution_payload": obj["executionPayload"],
            "share_state": obj["shareState"],
            "created_dt": cc.convert_epoch_millisecond_to_datetime(obj["created"]),
            "updated_dt": cc.convert_epoch_millisecond_to_datetime(obj["updated"]),
            "is_enabled": True if triggers_ls else False,
            "description": obj.get("jobDescription"),
            "accounts": obj["accounts"],
            "triggers": (
                [DomoTrigger._from_json(tg) for tg in triggers_ls]
                if triggers_ls
                else []
            ),
        }

    @classmethod
    def _from_json(
        cls,
        obj,
        auth,
    ):
        base_obj = cls._convert_API_res_to_DomoJob_base_obj(obj)

        return cls(
            auth=auth,
            **base_obj,
        )

    def _generate_to_json(self):
        return {
            "jobId": self.id,
            "jobName": self.name,
            "userId": self.user_id,
            "applicationId": self.application_id,
            "customerId": self.customer_id,
            "executionTimeout": self.execution_timeout,
            "executionPayload": self.execution_payload,
            "shareState": self.share_state,
            # created / excluded because generated metadata
            # updated / excluded because generated metadata
            "triggers": (
                [self.triggers[0].schedule.to_json()]
                if len(self.triggers) > 0
                else []
            ),
            "jobDescription": self.description,
            "executionTimeout": self.execution_timeout,
            "accounts": self.accounts,

        }

    def to_json(self):
        return self._generate_to_json()

In [8]:
#| exporti

@patch_to(DomoJob, cls_method=True)
async def _get_by_id(
    cls,
    application_id,
    job_id,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    debug_num_stacks_to_drop=2,
    new_cls: DomoJob = None, # pass in a child class which has the mandatory "from_json" function
    return_raw : bool = False,
    parent_class = None
):
    """
    this function will receive the parent_class as an input_parameter (instead of relying on the actual class DomoJob)
    to call the `new_class._from_json()`

    this process will handle converting the JSON obj into 'the correct' class 
    """

    res= await application_routes.get_application_job_by_id(
        auth=auth,
        application_id=application_id,
        job_id = job_id,
        session=session,
        debug_api=debug_api,
    
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res
            
    cls = new_cls or cls
    
    return cls._from_json(
        obj = res.response,
        auth = auth,
    )

@patch_to(DomoJob, cls_method= True)
async def get_by_id(
        cls,
        application_id,
        job_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        debug_num_stacks_to_drop=2,
        return_raw :bool = False,
    ):
        """
        stub abstract function that each `DomoJob` will have.
        note we pass the calling functions class into classmethod _get_by_id()
        so that we can call cls._from_json() during code execution 
        """
        
        return await cls._get_by_id(
            application_id = application_id,
            job_id = job_id,
            auth = auth,
            debug_api = debug_api,
            session = session,
            debug_num_stacks_to_drop= debug_num_stacks_to_drop,
            return_raw = return_raw,
            new_cls = cls
            )        
    

In [9]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

test_job = await DomoJob.get_by_id(
    job_id= '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
    application_id = 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
    auth = token_auth,
)

pprint(test_job)

DomoJob(id='743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
        name='tag_inactive_owners',
        user_id=1893952720,
        application_id='a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
        customer_id='mmmm-0012-0200',
        execution_timeout=1440,
        execution_payload={'domain': None,
                           'emails': ['jae@onyxreporting.com'],
                           'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5',
                           'ownerFilter': [],
                           'resourceTypes': ['datasource'],
                           'tagFilter': [],
                           'taggingPolicies': ['ownerInactive'],
                           'typeFilter': []},
        share_state={'sharedEntities': [{'accessLevel': 'OWNER',
                                         'id': '1893952720',
                                         'type': 'USER'}]},
        created_dt=datetime.datetime(2024, 2, 21, 11, 21, 31),
        updated_dt=datetime.datetime(2024, 2, 26

In [10]:
test_job.to_json()

{'jobId': '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
 'jobName': 'tag_inactive_owners',
 'userId': 1893952720,
 'applicationId': 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
 'customerId': 'mmmm-0012-0200',
 'executionTimeout': 1440,
 'executionPayload': {'emails': ['jae@onyxreporting.com'],
  'resourceTypes': ['datasource'],
  'tagFilter': [],
  'ownerFilter': [],
  'typeFilter': [],
  'domain': None,
  'taggingPolicies': ['ownerInactive'],
  'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5'},
 'shareState': {'sharedEntities': [{'id': '1893952720',
    'type': 'USER',
    'accessLevel': 'OWNER'}]},
 'triggers': [],
 'jobDescription': 't',
 'accounts': []}

# DomoJob Types
## DomoJob_RemoteDomoStats

In [11]:
# | exporti


@dataclass
class DomoJob_RemoteDomoStats(DomoJob):
    configured_reports: List[dict] = None
    remote_instance: str = None
    output_dataset_id: str = None
    subscriber_job_id: str = None

    @classmethod
    def _from_json(cls, obj, auth):

        return cls(
            **cls._convert_API_res_to_DomoJob_base_obj(obj),
            remote_instance=cls._get_remote_instance(
                obj["executionPayload"]["remoteInstance"]
            ),
            configured_reports=obj["executionPayload"]["policies"],
            output_dataset_id=obj["executionPayload"]["metricsDatasetId"],
            subscriber_job_id=obj["executionPayload"]["subscriberJobId"],
            auth=auth
        )

    @staticmethod
    def _get_remote_instance(remote_instance):
        return remote_instance.replace(".domo.com", "")

    @classmethod
    async def get_by_id(
        cls,
        application_id,
        job_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        debug_num_stacks_to_drop=2,
        return_raw: bool = False,
    ):

        return await cls._get_by_id(
            application_id=application_id,
            job_id=job_id,
            auth=auth,
            debug_api=debug_api,
            session=session,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            return_raw=return_raw,
            new_cls=cls,
            parent_class=cls.__name__,
        )

    def to_json(self):
        execution_payload = {
            "metricsDatasetId": self.output_dataset_id,
            "policies": self.configured_reports,
            "remoteInstance": self.remote_instance,
            "subscriberJobId": self.subscriber_job_id,
        }
        return {**self._generate_to_json(), "executionPayload": execution_payload}

In [12]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await application_routes.get_applications(auth=token_auth)

application_id = next(
    (app['applicationId'] for app in res.response if app["name"] == DomoJob_TypeEnum.REMOTE_DOMO_STATS.value)
)

res = await application_routes.get_application_jobs(
    application_id=application_id, auth=token_auth
)
job_id = res.response[0]['jobId']

test_job = await DomoJob_RemoteDomoStats.get_by_id(
    job_id= job_id,
    application_id = application_id,
    auth = token_auth,
    # return_raw = True
)

pprint(test_job)

DomoJob_RemoteDomoStats(id='f7a1b31f-f774-4eeb-ba85-d56d29b1dedc',
                        name='domo-community',
                        user_id=1893952720,
                        application_id='50e7230f-d2f2-42e2-a208-d94c8ae9f64c',
                        customer_id='mmmm-0012-0200',
                        execution_timeout=1440,
                        execution_payload={'metricsDatasetId': 'da68b3ea-088f-4493-ae69-ee5094feaa29',
                                           'policies': {'AccountPermissions': 'b156bfe4-d0e5-4410-8354-b3062973a99c'},
                                           'remoteInstance': 'domo-community',
                                           'subscriberJobId': 'd29c4551-5ec9-4eb8-8a93-b89e7a4ec300'},
                        share_state={'sharedEntities': [{'accessLevel': 'OWNER',
                                                         'id': '1893952720',
                                                         'type': 'USER'}]},
                       

In [13]:
pprint(test_job.to_json())

{'accounts': [96],
 'applicationId': '50e7230f-d2f2-42e2-a208-d94c8ae9f64c',
 'customerId': 'mmmm-0012-0200',
 'executionPayload': {'metricsDatasetId': 'da68b3ea-088f-4493-ae69-ee5094feaa29',
                      'policies': {'AccountPermissions': 'b156bfe4-d0e5-4410-8354-b3062973a99c'},
                      'remoteInstance': 'domo-community',
                      'subscriberJobId': 'd29c4551-5ec9-4eb8-8a93-b89e7a4ec300'},
 'executionTimeout': 1440,
 'jobDescription': 'updated on 2024-02-27',
 'jobId': 'f7a1b31f-f774-4eeb-ba85-d56d29b1dedc',
 'jobName': 'domo-community',
 'shareState': {'sharedEntities': [{'accessLevel': 'OWNER',
                                    'id': '1893952720',
                                    'type': 'USER'}]},
 'triggers': [{'eventEntity': '0 29 1 ? * *',
               'eventType': 'scheduleTriggered'}],
 'userId': 1893952720}


In [14]:
# | exporti
@patch_to(DomoJob_RemoteDomoStats, cls_method= True)
async def create(
    cls,
    auth: dmda.DomoAuth,
    trigger_schedule: DomoTrigger_Schedule,
    
    target_instance: str,
    output_dataset_id: str,
    account_id: str,
    
    report_dict: dict,
    
    execution_timeout: int = 1440,

    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: Optional[httpx.AsyncClient] = None,
):

    schedule_obj = trigger_schedule.to_json()

    body = application_routes.generate_body_remote_domostats(
        target_instance=target_instance,
        report_dict=report_dict,
        output_dataset_id=output_dataset_id,
        account_id=account_id,
        schedule_ls=[schedule_obj],
        execution_timeout=execution_timeout,
    )

    res = await application_routes.application(
        auth=auth,
        application_id=application_id,
        body=body,
        parent_class=cls.__name__,

        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    return cls._from_json(res.response, auth = auth)

NameError: name 'cls_method' is not defined

# DomoJob_Watchdog

In [ ]:
# | export


class Watchdog_Parameter_WatchType(Enum):
    ERROR_DETECTION = "error_detection"
    DATASET_INDEX_TIME = "max_indexing_time"
    LAST_UPDATED_DATA = "last_data_updated"
    ROW_COUNT_CHANGE = "row_count_variance"
    CUSTOM_QUERY = "custom_query"
    OUTLIER_EXECUTION = "execution_variance"


class Watchdog_Parameter_EntityType(Enum):
    DATAFLOW = "DATAFLOW"
    DATASET = "DATASET"


@dataclass
class Watchdog_Parameter:
    entity_ids: any
    entity_type: Watchdog_Parameter_EntityType
    watch_type: Watchdog_Parameter_WatchType

    @classmethod
    def _from_json(cls, obj):  # executionPayload

        obj_type = obj["type"].upper()
        entity_type = obj["entityType"].upper()
        entity_ids = obj["entityIds"]

        return cls(
            entity_ids=entity_ids,
            entity_type=Watchdog_Parameter_EntityType[entity_type],
            watch_type=Watchdog_Parameter_WatchType[obj_type],
        )

    def to_json(self):
        return {"entityIds": self.entity_ids,
                "entityType": self.entity_type.value,
                "type": self.watch_type.value}

In [ ]:
# | export


@dataclass
class DomoJob_Watchdog(DomoJob):
    custom_message: str = None
    remote_instance: str = None
    logs_dataset_id: str = None

    notify_emails: List[str] = None
    notify_group_ids: List[str] = None
    notify_user_ids: List[str] = None

    watcher_parameters: Watchdog_Parameter = None
    webhooks: List[str] = None

    @classmethod
    def _from_json(cls, obj, auth):

        remote_instance = obj["executionPayload"]["domain"]

        return cls(
            **cls._convert_API_res_to_DomoJob_base_obj(obj),
            custom_message = obj['executionPayload']['customMessage'],
            remote_instance=(
                cls._get_remote_instance(remote_instance) if remote_instance else None
            ),
            logs_dataset_id=obj["executionPayload"]["metricsDatasetId"],
            watcher_parameters=Watchdog_Parameter._from_json(
                obj["executionPayload"]["watcherParameters"]
            ),
            notify_emails=obj["executionPayload"]["notifyEmailAddresses"],
            notify_group_ids=obj["executionPayload"]["notifyGroupIds"],
            notify_user_ids=obj["executionPayload"]["notifyUserIds"],
            webhooks=obj["executionPayload"]["webhooks"],
            auth=auth
        )

    @staticmethod
    def _get_remote_instance(remote_instance):
        return remote_instance.replace(".domo.com", "")

    @classmethod
    async def get_by_id(
        cls,
        application_id,
        job_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        debug_num_stacks_to_drop=2,
        return_raw: bool = False,
    ):

        return await cls._get_by_id(
            application_id=application_id,
            job_id=job_id,
            auth=auth,
            debug_api=debug_api,
            session=session,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            return_raw=return_raw,
            new_cls=cls,
            parent_class=cls.__name__,
        )

    def to_json(self):
        execution_payload = {
            "customMessage": self.custom_message,
            "domain": self.remote_instance,
            "metricsDatasetId": self.logs_dataset_id,
            "notifyEmailAddresses": self.notify_emails,
            "notifyGroupIds": self.notify_group_ids,
            "notifyUserIds": self.notify_user_ids,
            "watcherParameters": self.watcher_parameters.to_json(),
            "webhooks": self.webhooks,
        }
        return {**self._generate_to_json(), "executionPayload": execution_payload}

In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-alpha",
    domo_access_token=os.environ["ALPHA_ACCESS_TOKEN"],
)

res = await application_routes.get_applications(auth=token_auth)

application_id = next(
    (app['applicationId'] for app in res.response if app["name"] == DomoJob_TypeEnum.DATA_WATCHDOG.value)
)

res = await application_routes.get_application_jobs(
    application_id=application_id, auth=token_auth
)
job_id = res.response[0]['jobId']

test_job = await DomoJob_Watchdog.get_by_id(
    job_id= job_id,
    application_id = application_id,
    auth = token_auth,
    # return_raw = True
)

pprint(test_job)

DomoJob_Watchdog(id='ca6bbd82-f0c7-4681-8946-5e6678179c07',
                 name='test',
                 user_id=1623162654,
                 application_id='33aab8f0-3397-45e1-933c-755abd0f5b3a',
                 customer_id='domo-alpha',
                 execution_timeout=1440,
                 execution_payload={'customMessage': '',
                                    'domain': None,
                                    'metricsDatasetId': '497edbb4-4041-4e39-a32c-d6e1923a0206',
                                    'notifyEmailAddresses': [],
                                    'notifyGroupIds': [],
                                    'notifyUserIds': [1623162654],
                                    'watcherParameters': {'entityIds': [None],
                                                          'entityType': 'DATAFLOW',
                                                          'type': 'error_detection'},
                                    'webhooks': []},
                 shar

In [ ]:
test_job.to_json()

{'jobId': 'ca6bbd82-f0c7-4681-8946-5e6678179c07',
 'jobName': 'test',
 'userId': 1623162654,
 'applicationId': '33aab8f0-3397-45e1-933c-755abd0f5b3a',
 'customerId': 'domo-alpha',
 'executionTimeout': 1440,
 'executionPayload': {'customMessage': '',
  'domain': None,
  'metricsDatasetId': '497edbb4-4041-4e39-a32c-d6e1923a0206',
  'notifyEmailAddresses': [],
  'notifyGroupIds': [],
  'notifyUserIds': [1623162654],
  'watcherParameters': {'entityIds': [None],
   'entityType': 'DATAFLOW',
   'type': 'error_detection'},
  'webhooks': []},
 'shareState': {'sharedEntities': [{'id': '1623162654',
    'type': 'USER',
    'accessLevel': 'OWNER'}]},
 'triggers': [],
 'jobDescription': 'updated on 2024-02-27',
 'accounts': []}

# Domo Job CRUD Operations

In [ ]:

@patch_to(DomoJob)
async def update(
    self : DomoJob ,
    body: dict = None,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    debug_num_stacks_to_drop = 2
):
    
    res = await application_routes.update_job(
        auth=self.auth,
        body=body or self.to_json(),
        application_id=self.application_id,
        job_id=self.id,
        debug_api=debug_api,
        session=session,
        parent_class = self.__class__.__name__,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop
    )

    return res

#### sample update

In [ ]:
test_job.description = f"updated on {dt.date.today()}"

await test_job.update()

ResponseGetData(status=200, response={'jobId': 'ca6bbd82-f0c7-4681-8946-5e6678179c07', 'applicationId': '33aab8f0-3397-45e1-933c-755abd0f5b3a', 'customerId': 'domo-alpha', 'jobName': 'test', 'jobDescription': 'updated on 2024-02-27', 'userId': 1623162654, 'executionTimeout': 1440, 'jobStatus': 'idle', 'executionPayload': {'webhooks': [], 'domain': None, 'watcherParameters': {'entityIds': [None], 'entityType': 'DATAFLOW', 'type': 'error_detection'}, 'customMessage': '', 'metricsDatasetId': '497edbb4-4041-4e39-a32c-d6e1923a0206', 'notifyUserIds': [1623162654], 'notifyEmailAddresses': [], 'notifyGroupIds': []}, 'executionResponse': {}, 'accounts': [], 'executionClass': 'com.domo.executor.datawatchdog.DataWatchdogExecutor', 'created': 1709058965000, 'updated': 1709077199000, 'statusChanged': 1709076890000, 'triggers': [], 'compressPayload': False}, is_success=True, parent_class=None)

In [ ]:
# |exporti

@patch_to(DomoJob)
async def execute(
    self,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    debug_num_stacks_to_drop = 2
):

    res = await application_routes.execute_application_job(
        auth=self.auth,
        application_id=self.application_id,
        job_id=self.id,
        debug_api=debug_api,
        session=session,
        parent_class = self.__class__.__name__,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop

    )

    return res

#### sample execute

In [ ]:
await test_job.execute()

ResponseGetData(status=200, response={'requestedBy': 1623162654, 'queuedJobs': [{'jobId': 'ca6bbd82-f0c7-4681-8946-5e6678179c07', 'applicationId': '33aab8f0-3397-45e1-933c-755abd0f5b3a', 'executionId': 'a7ff1299-0d28-4161-a47e-963bb72a4324', 'customerId': 'domo-alpha', 'jobName': 'test', 'jobDescription': 'updated on 2024-02-27', 'userId': 1623162654, 'executionTimeout': 1440, 'jobStatus': 'idle', 'executionPayload': {'webhooks': [], 'domain': None, 'watcherParameters': {'entityIds': [None], 'entityType': 'DATAFLOW', 'type': 'error_detection'}, 'customMessage': '', 'metricsDatasetId': '497edbb4-4041-4e39-a32c-d6e1923a0206', 'notifyUserIds': [1623162654], 'notifyEmailAddresses': [], 'notifyGroupIds': []}, 'executionResponse': {}, 'accounts': [], 'executionClass': 'com.domo.executor.datawatchdog.DataWatchdogExecutor', 'created': 1709058965000, 'updated': 1709077199000, 'statusChanged': 1709076890000, 'triggers': [], 'compressPayload': False}]}, is_success=True, parent_class=None)

In [ ]:


@classmethod
async def create_watchdog_job(
    cls,
    auth: dmda.DomoFullAuth,
    body: str,
    application_id: str,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
):

    res = await application_routes.add_job(
        auth=auth,
        application_id=application_id,
        body=body,
        debug_api=debug_api,
        session=session,
    )
    if debug_api:
        print(res)

    if res.status != 200:
        return False

    return True

@classmethod
async def generate_watchdog_body(
    cls,
    watchdog_report_type: WatchDogType,
    watchdog_schedule: DomoTrigger_Schedule,
    job_name: str,
    notify_user_ids_ls: list,
    notify_group_ids_ls: list,
    notify_emails_ls: list,
    entity_ids_ls: list,
    entity_type: str,
    metric_dataset_id: str,
    sql_query: str = None,
    variance_percent: int = 30,
    max_indexing_time_mins: int = 30,
    execution_timeout: int = 1440,
    min_update_frequency_min: int = 1440,
):
    schedule_obj = watchdog_schedule.to_json()

    body = application_routes.generate_body_watchdog_generic(
        job_name=job_name,
        notify_user_ids_ls=notify_user_ids_ls,
        notify_group_ids_ls=notify_group_ids_ls,
        notify_emails_ls=notify_emails_ls,
        entity_ids_ls=entity_ids_ls,
        entity_type=entity_type,
        metric_dataset_id=metric_dataset_id,
        schedule_ls=[schedule_obj],
        job_type=watchdog_report_type.value,
        execution_timeout=execution_timeout,
    )

    if watchdog_report_type == watchdog_report_type.DATASET_INDEX_TIME:
        child = {"maxIndexingTimeInMinutes": max_indexing_time_mins}
        body["executionPayload"]["watcherParameters"].update(child)

    if (
        watchdog_report_type == watchdog_report_type.ROW_COUNT_CHANGE
        or watchdog_report_type == watchdog_report_type.OUTLIER_EXECUTION
    ):
        child = {"variancePercent": variance_percent}
        body["executionPayload"]["watcherParameters"].update(child)

    if watchdog_report_type == watchdog_report_type.CUSTOM_QUERY:
        child = {"sqlQuery": sql_query}
        body["executionPayload"]["watcherParameters"].update(child)

    if watchdog_report_type == watchdog_report_type.LAST_UPDATED_DATA:
        child = {"minDataUpdateFrequencyInMinutes": min_update_frequency_min}
        body["executionPayload"]["watcherParameters"].update(child)

    return body



NameError: name 'WatchDogType' is not defined